In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip install implicit

In [3]:
data = pd.read_csv('./retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items, items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    # your_code
    # ВАЖНО: я изменил интерфейс функции. Считаю, туда должны явно передаваться items.
    items = np.array(items)
    items_weights = np.array(items_weights)
    recs = np.random.choice(items, size=n, p=items_weights,replace=False)
    
    
    
    return recs.tolist()

In [6]:
%%time

# your_code
popularity = data_train.groupby('item_id', as_index=False)['sales_value'].sum()
popularity['weights'] = popularity.sales_value / popularity.sales_value.sum()
popularity.drop(columns=['sales_value'], inplace=True)
data_train = data_train.merge(popularity, on='item_id')

result = pd.read_csv('./predictions_basic.csv')

items = data_train.item_id.unique()
items_weights = data_train.drop_duplicates(subset='item_id').weights
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation
                                                                   (items=items, items_weights=items_weights, n=5))
result.head(2)

CPU times: total: 3.8 s
Wall time: 3.8 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[997821, 12302069, 1099058, 935646, 996269]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[982469, 9655679, 976335, 938074, 12263759]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [7]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[997821, 12302069, 1099058, 935646, 996269]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[982469, 9655679, 976335, 938074, 12263759]"


In [8]:
# your_code

Сначала напишу функцию для расчёта precision@k:

In [9]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]  
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)    
    
    return precision


Потом путем проб и ошибок можно обратить внимание на то, что данные даны в совершенно несъедобном формате: строка, в ней список строк, а уже в них значения. Не надо так.  
Напишу функцию для преобразования данных к читаемому виду. 

In [10]:
def gay_style_data_to_normal(gay_style_data: str) -> list:
    lst = gay_style_data.split()
    normal_data = []
    for el in lst:
        tmp = ''
        for ch in el:
            if ch.isnumeric():
                tmp += ch
        normal_data.append(int(tmp))
    return normal_data

Преобразуем данные

In [11]:
cols = ['actual', 'random_recommendation', 'popular_recommendation', 
        'itemitem', 'cosine', 'tfidf', 'own_purchases',
       'weighted_random_recommendation']

In [12]:
for col in cols:
    try:
        result[col] = result[col].apply(gay_style_data_to_normal)
        print(f'cured col {col}')
    except:
        print(f'unable to cure col {col}')

unable to cure col actual
cured col random_recommendation
cured col popular_recommendation
cured col itemitem
cured col cosine
cured col tfidf
cured col own_purchases
unable to cure col weighted_random_recommendation


Признак actual был законспирирован другим образом, не буду писать для него отдельную функцию, а преобразую на лету.

In [13]:
result.actual = result.actual.apply(lambda x: [int(item) for item in x.split() if item.isnumeric()])
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[997821, 12302069, 1099058, 935646, 996269]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[982469, 9655679, 976335, 938074, 12263759]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]","[914318, 916590, 859075, 1129576, 880664]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]","[9527183, 1091021, 877723, 972143, 1074981]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]","[958661, 1094767, 965208, 1029479, 1098189]"


Теперь все готово, можно считать precision@5

In [14]:
for col in cols:
    try:
        metric = result.apply(lambda row: precision_at_k(row[col], row['actual']), axis=1).mean()
        if col != 'actual':
            print(f'metric for {col}: {metric:.4f}')
    except:
        print(f'unable to count metric for {col}')

metric for random_recommendation: 0.0005
metric for popular_recommendation: 0.1485
metric for itemitem: 0.0329
metric for cosine: 0.0348
metric for tfidf: 0.0357
metric for own_purchases: 0.1739
metric for weighted_random_recommendation: 0.0207
